In [1]:
import deeplabcut as dlc
import random
import shutil
import glob
import os

# From paths.py
import paths

C:\Users\mmccann\Miniconda3\envs\DLC-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mmccann\Miniconda3\envs\DLC-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mmccann\Miniconda3\envs\DLC-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mmccann\Miniconda3\envs\DLC-GPU\lib\site-packages

In [2]:
# Find all videos in the data directory with VTuning in the name
all_videos = glob.glob(paths.vrexperiment_path + os.sep + "*VTuning*.avi")

# If Project already created, run the cell below

In [3]:
# Get config path
config_path = paths.config_path_VTuning

# Read config file
cfg = dlc.auxiliaryfunctions.read_config(config_path)

# Get project directory
project_directory = cfg['project_path']

# Get list of local videos as found in the config
project_video_list = [vid for vid in cfg["video_sets"]]
test_video_dir = os.path.join(project_directory, "test-videos")
test_vids = glob.glob(test_video_dir + os.sep + "*VWheel*.avi")

# If you haven't created a new project, run this cell

In [ ]:
dlc.create_new_project?

In [ ]:
# Randomly select 15 videos to train on
remote_video_list = []
for i in range(20):
    remote_video_list.append(random.choice(all_videos))

dlc.create_new_project("VTuning", "Matt", remote_video_list, working_directory=r"D:\DLC_projects\vtuning_wired", copy_videos=True)

In [ ]:
# Get config and path info
config_path = paths.config_path_VTuning
cfg = dlc.auxiliaryfunctions.read_config(config_path)
project_directory = cfg['project_path']
project_video_list = [vid for vid in cfg["video_sets"]]

In [ ]:
# Randomly pick videos to test the network that are not part of the training set and copy to the project directory
test_video_dir = os.path.join(project_directory, "test-videos")
if not os.path.isdir(test_video_dir):
    os.mkdir(test_video_dir)

# Randomly pick five videos outside of the training set to test the network on
test_vids = []
while(len(test_vids) < 5):
    vid = random.choice(all_videos)
    if vid not in project_video_list:
        dest = shutil.copy2(vid, test_video_dir)
        test_vids.append(dest)
        
test_vids = glob.glob(test_video_dir + os.sep + "*VWheel*.avi")

# Add extra videos (if needed)

In [ ]:
dlc.add_new_videos?

In [ ]:
# If adding new videos, do that here by randomly selecting some not in the original data set or the test set
# For this particular case, we want videos that are older than March 12th 2022
from datetime import datetime
from os.path import sep

valid_extras = [vid for vid in all_videos if datetime.strptime(vid.split(sep)[-1].split('_VTuning')[0], "%m_%d_%Y_%H_%M_%S") > datetime(2022, 3, 12)]

extra_vids = []
while(len(extra_vids) < 5):
    vid = random.choice(valid_extras)
    if vid not in project_video_list or test_vids:
        extra_vids.append(vid)
        
dlc.add_new_videos(config_path, extra_vids, copy_videos=True)

# Extract and label frames

In [ ]:
dlc.extract_frames?

In [ ]:
# extract frames from the videos
# will go through all of the videos on the list, so if many videos, it might take a while
dlc.extract_frames(config_path, mode='automatic', algo='kmeans', crop=False, userfeedback=True);

In [ ]:
# label frames
# generates folders in the labeled-data folder with the video name
dlc.label_frames(config_path)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


In [ ]:
# check the labels
# generates folders in the labeled-data folder with the video name but the actual labels printed on the pics
dlc.check_labels(config_path, draw_skeleton=False);

In [5]:
# Used if need to |reconstruct labeling h5 files from csvs
dlc.convertcsv2h5(config_path)

Do you want to convert the csv file in folder: D:\DLC_projects\vtuning_wired\VTuning-Matt-2022-04-06\labeled-data\02_25_2022_15_16_33_VTuning_MM_220121_a_free0 ?
yes/noy
Do you want to convert the csv file in folder: D:\DLC_projects\vtuning_wired\VTuning-Matt-2022-04-06\labeled-data\02_22_2022_11_18_44_VTuning_MM_220117_a_free1 ?
yes/noy
Do you want to convert the csv file in folder: D:\DLC_projects\vtuning_wired\VTuning-Matt-2022-04-06\labeled-data\03_01_2022_16_06_35_VTuning_MM_220120_a_free1 ?
yes/noy
Do you want to convert the csv file in folder: D:\DLC_projects\vtuning_wired\VTuning-Matt-2022-04-06\labeled-data\02_22_2022_12_21_28_VTuning_MM_220118_a_free2 ?
yes/noy
Do you want to convert the csv file in folder: D:\DLC_projects\vtuning_wired\VTuning-Matt-2022-04-06\labeled-data\03_04_2022_13_28_55_VTuning_MM_220118_a_free1 ?
yes/noy
Do you want to convert the csv file in folder: D:\DLC_projects\vtuning_wired\VTuning-Matt-2022-04-06\labeled-data\03_02_2022_12_21_03_VTuning_MM_22011

In [6]:
# create the training set
# also creates the Pose.yaml in the dlc-models/train dataset, which contains the setting for training, i.e. take a look
dlc.create_training_dataset(config_path, augmenter_type='default');

D:\DLC_projects\vtuning_wired\VTuning-Matt-2022-04-06\training-datasets\iteration-1\UnaugmentedDataSet_VTuningApr6  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


In [7]:
# train the network
dlc.train_network(config_path, max_snapshots_to_keep=10, displayiters=50000, saveiters=50000)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11]],
 'all_joints_names': ['mouseSnout',
                      'mouseBarL',
                      'mouseBarR',
                      'mouseBody1',
                      'mouseBody2',
                      'mouseBody3',
                      'mouseBase',
                      'miniscope',
                      'corner_UL',
                      'corner_UR',
                      'corner_BR',
                      'corner_BL'],
 'alpha_r': 0.02,
 'batch_size': 1,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_VTuningApr6\\VTuning_Matt95shuffle1.mat',
 'dataset_type': 'default',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\mmccann\\Miniconda3\\envs\\DLC-GPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 '

Selecting single-animal trainer
Starting with imgaug pose-dataset loader (=default).
Batch Size is 1
Initializing ResNet
Loading ImageNet-pretrained resnet_50
Display_iters overwritten as 50000
Save_iters overwritten as 50000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'D:\\DLC_projects\\vtuning_wired\\VTuning-Matt-2022-04-06\\dlc-models\\iteration-1\\VTuningApr6-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'default', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_

iteration: 50000 loss: 0.0055 lr: 0.02
iteration: 100000 loss: 0.0024 lr: 0.02
iteration: 150000 loss: 0.0019 lr: 0.02
iteration: 200000 loss: 0.0016 lr: 0.02
iteration: 250000 loss: 0.0015 lr: 0.02
iteration: 300000 loss: 0.0014 lr: 0.02
iteration: 350000 loss: 0.0013 lr: 0.02
iteration: 400000 loss: 0.0012 lr: 0.02
iteration: 450000 loss: 0.0011 lr: 0.002
iteration: 500000 loss: 0.0009 lr: 0.002
iteration: 550000 loss: 0.0009 lr: 0.002
iteration: 600000 loss: 0.0009 lr: 0.002
iteration: 650000 loss: 0.0009 lr: 0.002
iteration: 700000 loss: 0.0009 lr: 0.002
iteration: 750000 loss: 0.0009 lr: 0.001
iteration: 800000 loss: 0.0009 lr: 0.001
iteration: 850000 loss: 0.0008 lr: 0.001
iteration: 900000 loss: 0.0008 lr: 0.001
iteration: 950000 loss: 0.0008 lr: 0.001
iteration: 1000000 loss: 0.0008 lr: 0.001


The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\mmccann\Miniconda3\envs\DLC-GPU\lib\site-packages\tensorflow\python\client\session.py", line 1334, in _do_call
    return fn(*args)
  File "C:\Users\mmccann\Miniconda3\envs\DLC-GPU\lib\site-packages\tensorflow\python\client\session.py", line 1319, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "C:\Users\mmccann\Miniconda3\envs\DLC-GPU\lib\site-packages\tensorflow\python\client\session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_enqueue}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mmccann\Miniconda3\envs\DLC-GPU\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\mmccann\Miniconda3\envs\DLC-GPU\lib\threading.py", line 870, in run
    self._targ

In [8]:
# evaluate the network
dlc.evaluate_network(config_path, Shuffles=[1], plotting=True)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11]],
 'all_joints_names': ['mouseSnout',
                      'mouseBarL',
                      'mouseBarR',
                      'mouseBody1',
                      'mouseBody2',
                      'mouseBody3',
                      'mouseBase',
                      'miniscope',
                      'corner_UL',
                      'corner_UR',
                      'corner_BR',
                      'corner_BL'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_VTuningApr6\\VTuning_Matt95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\mmccann\\Miniconda3\\envs\\DLC-GPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_r

D:\DLC_projects\vtuning_wired\VTuning-Matt-2022-04-06/evaluation-results/  already exists!
Running  DLC_resnet_50_VTuningApr6shuffle1_1030000  with # of trainingiterations: 1030000
Initializing ResNet


0it [00:00, ?it/s]

Analyzing data...


509it [00:57,  8.87it/s]
  0%|                                                                                          | 0/509 [00:00<?, ?it/s]

Done and results stored for snapshot:  snapshot-1030000
Results for 1030000  training iterations: 95 1 train error: 1.97 pixels. Test error: 5.75  pixels.
With pcutoff of 0.6  train error: 1.86 pixels. Test error: 4.6 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


100%|████████████████████████████████████████████████████████████████████████████████| 509/509 [02:28<00:00,  3.44it/s]

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


In [10]:
dlc.analyze_videos(config_path, test_video_dir)
dlc.create_labeled_video(config_path, test_video_dir)

Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11]],
 'all_joints_names': ['mouseSnout',
                      'mouseBarL',
                      'mouseBarR',
                      'mouseBody1',
                      'mouseBody2',
                      'mouseBody3',
                      'mouseBase',
                      'miniscope',
                      'corner_UL',
                      'corner_UR',
                      'corner_BR',
                      'corner_BL'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets\\iteration-1\\UnaugmentedDataSet_VTuningApr6\\VTuning_Matt95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\mmccann\\Miniconda3\\envs\\DLC-GPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_r

Using snapshot-1030000 for model D:\DLC_projects\vtuning_wired\VTuning-Matt-2022-04-06\dlc-models\iteration-1\VTuningApr6-trainset95shuffle1
Initializing ResNet
No video(s) were found. Please check your paths and/or 'video_type'.
No video(s) were found. Please check your paths and/or 'video_type'.


In [ ]:
dlc.extract_outlier_frames?

In [ ]:
# extract outlier frames
# dlc.extract_outlier_frames(config_path, project_video_list, outlieralgorithm='uncertain', p_bound=0.6)
dlc.extract_outlier_frames(config_path, project_video_list)

In [ ]:
dlc.refine_labels(config_path)

In [ ]:
# Merge the dataset, then move to check labels and retrain
dlc.merge_datasets(config_path)

In [ ]:
dlc.analyze_videos(config_path, test_vids)
dlc.create_labeled_video(config_path, test_vids)